# VariantSpark-k Medium Parquet
 - 5,000 samples  
 - 100,000,000 variables   
 - ?? minutes to run

In [ ]:
%%bash

set -e

MASTER=https://kubernetes.default.svc:443
INPUT_BUCKET=variant-spark

function fatal_error () {
	echo "ERROR: $1" 1>&2
	exit 1
}

if [ -z ${MASTER+x} ];
    then
        echo "You must set the MASTER environment variable to a kubernetes API endpoint";
        echo "Example: https://ABC.sk1.us-west-2.eks.amazonaws.com:443"
        exit 1
fi

if [ -z ${INPUT_BUCKET+x} ];
    then
        echo "You must set the INPUT_BUCKET environment variable to a bucket containing input data";
        echo "Example: variant-spark-k-storage"
        exit 1
fi

[[ $(type -P "spark-submit") ]] || fatal_error  "\`spark-submit\` cannot be found. Please make sure it's on your PATH."

spark-submit \
    --class au.csiro.variantspark.cli.VariantSparkApp \
    --driver-class-path ./conf \
    --master k8s://${MASTER} \
    --deploy-mode cluster \
    --name VariantSparkMediumParquet \
    --conf spark.kubernetes.authenticate.driver.serviceAccountName=spark \
    --conf spark.executor.instances=28 \
    --conf spark.executor.memory=4g \
    --conf spark.kubernetes.container.image=jamesrcounts/variantspark:002 \
    --jars http://central.maven.org/maven2/org/apache/hadoop/hadoop-aws/2.7.3/hadoop-aws-2.7.3.jar,http://central.maven.org/maven2/com/amazonaws/aws-java-sdk/1.7.4/aws-java-sdk-1.7.4.jar,http://central.maven.org/maven2/joda-time/joda-time/2.9.9/joda-time-2.9.9.jar \
    local:///opt/spark/jars/variant-spark_2.11-0.2.0-SNAPSHOT-all.jar importance \
        -if s3a://${INPUT_BUCKET}/datasets/synthetic/data_s5000_v100000000_r13.parquet \
        -ff s3a://${INPUT_BUCKET}/datasets/synthetic/labels_s5000_v100000000_r13_f0.125.csv \
        -fc resp \
        -it parquet
        -v \
        -rn 500 \
        -rbs 28 \
        -ro "$@"
